# Inferring the goals of autonomous agents in Gen.jl

what is Gen.jl (2 sentences)

In [1]:
using Gen

## 1. Modeling an autonomous agent

Explain ( 2 sentence). Explain that a model is a probabilisttic program, and that @program is used to define a probabilistic program in Gen.jl

In [2]:
include("scene.jl")
include("path_planner.jl");

In [3]:
@program agent_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
    add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
    add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
    add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
    add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))    
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

We can run the model to generate probable scenarios:. We run a model using `@generate`, which executes a program and populates a trace with the values of the named expressions in the program.

EXPLAIN THAT ANY PROGRAM STATE CAN BE TRACED INCLUDING RANDOM CHOICES AND NON-RANDOM CHOICES

In [4]:
trace = Trace()
@generate(trace, agent_model())
println("start: ", value(trace, "start"))
println("destination: ", value(trace, "destination"))
println("x1 through x4: ", map((i) -> value(trace, "x$i"), 1:4))
println("y1 through y4: ", map((i) -> value(trace, "y$i"), 1:4))

start: Point(41.62377528866601,6.023750933344041)
destination: Point(93.58995599592403,64.12110725135756)
x1 through x4: [38.9629,47.3418,54.0598,64.667]
y1 through y4: [5.05266,15.9794,21.2188,28.5684]


If we run it again, we get a different result:

In [5]:
trace = Trace()
@generate(trace, agent_model())
println("start: ", value(trace, "start"))
println("destination: ", value(trace, "destination"))
println("x1 through x4: ", map((i) -> value(trace, "x$i"), 1:4))
println("y1 through y4: ", map((i) -> value(trace, "y$i"), 1:4))

start: Point(41.39195239154767,86.56500755895678)
destination: Point(0.030669177779141243,46.42609827425972)
x1 through x4: [40.8192,32.3991,20.6858,14.8003]
y1 through y4: [86.5162,84.6837,81.4562,72.2509]


We can view the whole trace. For now, don't worry about constraints, interventions, or proposals. Note that the `recorded` section lists all the values that were recorde. 

In [297]:
print(trace)

-- Constraints --
x2 => 10
y2 => 20
x3 => 10
y3 => 30
-- Interventions --
start => Point(10.0,10.0)
destination => Point(90.0,90.0)
-- Proposals --
-- Recorded --
times => [0.0,10.5263,21.0526,31.5789,42.1053,52.6316,63.1579,73.6842,84.2105,94.7368,105.263,115.789,126.316,136.842,147.368,157.895,168.421,178.947,189.474,200.0]
x1 => 8.83997947888003
y1 => 11.341483896885006
x4 => 35.33840953247626
y4 => 26.35067079865551
x5 => 46.57803086936879
y5 => 27.070667396525064
x6 => 56.619593824570806
y6 => 26.825819036207797
x7 => 66.21799347978964
y7 => 27.852266426999197
x8 => 74.8121642921307
y8 => 30.00755015282657
x9 => 85.30058858539059
y9 => 35.6151566249177
x10 => 86.38245413317341
y10 => 47.281996671419456
x11 => 85.55774145581165
y11 => 59.36349445613278
x12 => 88.9165208250659
y12 => 66.87120768182838
x13 => 88.53546062544528
y13 => 75.98136445763129
x14 => 90.41550187974406
y14 => 88.85363034834937
x15 => 89.52756660132378
y15 => 89.35750534627498
x16 => 90.38749918925689
y16 => 89

# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing out the values of variables is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. In Gen.jl a trace renderer is simply object that has a method `render(trace::Trace)`. In Jupyter notebooks, we render traces using JavaScript code with a generic `JupyterInlineRenderer`. 

FIX ME


In [389]:
javascript"""

var Gen = require("nbextensions/gen_notebook_extension/main");
var d3 = require("nbextensions/d3/d3.min");

Gen.register_jupyter_renderer("agent_model_renderer", function(id, trace) {
    
    var root = d3.select("#" + id);
    
    // the base of the scene where all data is rendered
    var svg = root.selectAll("svg").data([""]);
    svg = svg.enter().append("svg")
        .attr("viewBox", "0 0 100 100")
        .attr("position", "absolute")
        .style("height", "100%")
        .merge(svg);
    
    // bounding box around the frame
    svg.selectAll("rect").data([""]).enter().append("rect")
        .attr("width", "100%")
        .attr("height", "100%")
        .attr("stroke", "#000")
        .attr("fill", "#FFF");
    
    // the path points of the agent
    // TODO would be great if they could be observed together..
    var times = Gen.find_choice(trace, "times").value;
    var path_point_data = [];
    for (var i=1; i<=times.length; i++) {
        path_point_data.push({x: Gen.find_choice(trace, "x" + i),
                              y: Gen.find_choice(trace, "y" + i)});
    }
    var path_segment_data = [];
    for (var i=0; i<times.length-1; i++) {
        path_segment_data.push({prev: {x: path_point_data[i].x,   y: path_point_data[i].y},
                                next: {x: path_point_data[i+1].x, y: path_point_data[i+1].y}});
    }
    var radius = 2;
    var path_points = svg.selectAll(".path").data(path_point_data);
    path_points.exit().remove();
    path_points.enter().append("circle")
        .attr("r", radius)
        .style("fill", "orange")
        .style("fill-opacity", 0.5)
        .classed("path", true)
      .merge(path_points)
        .attr("cx", function(d) { return d.x.value; })
        .attr("cy", function(d) { return d.y.value; });
    svg.selectAll(".path")
        .classed("interventions", function(d) { return d.x.where == Gen.interventions || d.y.where == Gen.interventions; })
        .classed("constraints", function(d) { return d.x.where == Gen.constraints || d.y.where == Gen.constraints; });
    
    var path_segments = svg.selectAll(".path_segments").data(path_segment_data);
    path_segments.exit().remove();
    path_segments.enter().append("line")
        .style("stroke", "#000")
        .classed("path_segments", true)
      .merge(path_segments)
        .attr("x1", function(d) { return d.prev.x.value; })
        .attr("y1", function(d) { return d.prev.y.value; })
        .attr("x2", function(d) { return d.next.x.value; })
        .attr("y2", function(d) { return d.next.y.value; });
    
    // the starting position of the agent
    var trace_start = Gen.find_choice(trace, "start");
    var start = svg.selectAll(".start").data(trace_start ? [trace_start] : []);
    start.exit().remove();
    start.enter().append("circle")
        .attr("r", radius)
        .style("fill", "blue")
        .classed("start", true)
      .merge(start)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });
    
    // the destination position of the agent
    var trace_dest = Gen.find_choice(trace, "destination");
    var dest = svg.selectAll(".destination").data(trace_dest ? [trace_dest] : []);
    dest.exit().remove();
    dest.enter().append("circle")
        .attr("r", radius)
        .style("fill", "red")
        .classed("destination", true)
      .merge(dest)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });

    // whether the start and destination are intervened or constrained
    svg.selectAll(".destinations, .start")
        .classed("interventions", function(d) { return d.where == Gen.interventions; })
        .classed("constraints", function(d) { return d.where == Gen.constraints; });

    // apply styles to indicate intervened or constrained
    svg.selectAll(".interventions")
        .style("stroke", "#000")
        .style("stroke-width", 2);
    svg.selectAll(".constraints")
        .style("stroke", "#000")
        .style("stroke-width", 2)
        .style("stroke-dasharray", "1, 1");
    
    // make the legend
    // TODO
    
    
    // Draw the obstacles in the scene
    
    var trace_scene = Gen.find_choice(trace, "scene");
    
    var trace_trees = trace_scene.value.obstacles.filter(function(element) { return element.name == "Tree";});
    var trees = svg.selectAll(".tree").data(trace_trees);
    trees.exit().remove();
    trees.enter().append("rect")
        .style("fill", "green")
        .classed("tree", true)
      .merge(trees)
        .attr("x", function(d) { return d.center.x - d.size/2.0; })
        .attr("y", function(d) { return d.center.y - d.size/2.0; })
        .attr("width", function(d) { return d.size; })
        .attr("height", function(d) { return d.size; });
    
    var trace_walls = trace_scene.value.obstacles.filter(function(element) { return element.name == "Wall";});
    var walls = svg.selectAll(".wall").data(trace_walls);
    walls.exit().remove();
    walls.enter().append("rect")
        .style("fill", "gray")
        .classed("wall", true)
      .merge(walls)
        .attr("x", function(d) { return d.start.x; })
        .attr("y", function(d) { return d.start.y; })
        .attr("width", function(d) { return d.orientation == 1 ? d.length : d.thickness; })
        .attr("height", function(d) { return d.orientation == 2 ? d.length : d.thickness; });
    
    // TODO add the log score
    var score = svg.selectAll(".score").data([""]);
    score.enter().append("text")
        .classed("score", true)
        .attr("x", 50).attr("y", 95).attr("text-anchor", "middle")
        .attr("font-size", "10px")
      .merge(score)
        .text(trace.log_weight.toFixed(2))
    
});
"""

In [390]:
renderer = JupyterInlineRenderer("agent_model_renderer")
viewport(renderer)

HTML{String}("<div style=\"border-style: solid; border-radius: 25px; text-align: center;\">\nTrace Rendering (agent_model_renderer)\n<div id=\"id_MVIoFDb6mj92O5cTJgOt\" style=\"height: 200px; resize: vertical; position: relative; overflow: hidden; text-align: center;\">\n</div>\n</div>\n    ")

Let's use this rendering to visualize a run of the our program. It will be visualized in the rendering view above.

In [391]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
intervene!(trace, "destination", Point(90, 90))
constrain!(trace, "x2", 10)
constrain!(trace, "y2", 20)
constrain!(trace, "x3", 10)
constrain!(trace, "y3", 30)
for i=1:1
    @generate(trace, agent_model())
    render(renderer, trace)
end

Since the behavior is stochastic, we need to visualize many samples at once in a grid, to get a sense of the full distribution:

In [9]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 3, 1, 900, 300)
inline(tiled)

LoadError: UndefVarError: inline not defined

In [10]:
traces = [begin trace = Trace(); @generate(trace, agent_model()); trace end for i=1:3]
render(tiled, traces)

LoadError: No target has been defined

We can intervene and see what the traces look like when we fix the value of `start` and `destination` to specific values:

In [11]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(tiled)

LoadError: UndefVarError: inline not defined

In [12]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
intervene!(trace, "destination", Point(90, 90))
traces = Trace[]
for i=1:27
    t = deepcopy(trace)
    @generate(t, agent_model())
    push!(traces, t)
end
render(tiled, traces)

LoadError: No target has been defined

# 3. Probabilistic inference

So far, we have simulated forward from the model, and we have intervened on some random choices and simulated the consequence. However, suppose we had observed a given sequence of locations of the agent, and we wanted to know probable goal locations? This is a query that cannot be answered simply by forward simulation of the program, because the location of the drone is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 15 time points:

In [13]:
points = [Point(10, 10), Point(10, 20), Point(10, 30), Point(10, 40)]

4-element Array{Point,1}:
 Point(10.0,10.0)
 Point(10.0,20.0)
 Point(10.0,30.0)
 Point(10.0,40.0)

The first step in inference is to constrain the random choices that are observed.

In [14]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

Now, when we run the program in this trace, we find that the score of the trace tells us how well the trace matches the constraints.

In [15]:
tiled = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(tiled)

LoadError: UndefVarError: inline not defined

In [16]:
traces = Trace[]
for i=1:27
    t = deepcopy(trace)
    @generate(t, agent_model())
    push!(traces, t)
end
render(tiled, traces)

LoadError: No target has been defined

### Importance sampling

We can use this score to filter out traces that don't match well with the observations. Specifically, we can sample a large number of traces, and pick one in proportion to the exponential of its score. We can then repreat this whole process a number of times:

In [17]:
# TODO create an HTML helper that creates an array of SVG elements, that can then be filled in one by one
# while the algorithm is running.
importance_sampling_result_renderer = TiledJupyterInlineRenderer("agent_model_renderer", 9, 3, 900, 300)
inline(importance_sampling_result_renderer)

LoadError: UndefVarError: inline not defined

In [18]:
function logsumexp(arr::Vector{Float64})
    min_arr = maximum(arr)
    min_arr + log(sum(exp(arr - min_arr)))
end

approximate_samples = Trace[]
num_approximate_samples = 27
num_simulations = 10
for i=1:num_approximate_samples
    traces = Vector{Trace}(num_simulations)
    scores = Vector{Float64}(num_simulations)
    for i=1:num_simulations
        t = deepcopy(trace)
        @generate(t, agent_model())
        scores[i] = score(t)
        traces[i] = t
    end
    weights = exp(scores - logsumexp(scores))
    weights = weights / sum(weights)
    chosen = rand(Categorical(weights))
    push!(approximate_samples, traces[chosen])
end
# TODO render incrementally!
render(importance_sampling_result_renderer, approximate_samples)

LoadError: No target has been defined

In [19]:
# show compositing of the goal rendering...


In [20]:
# show SIR results in a tile plots

### Metropolis-Hastings Inference

In [21]:
current_trace_renderer = JupyterInlineRenderer("agent_model_renderer")
attach(current_trace_renderer, "current");
proposed_trace_renderer = JupyterInlineRenderer("agent_model_renderer")
attach(proposed_trace_renderer, "proposed");
HTML("""
        <svg id="current" height="200" viewBox="0 0 100 100"></svg>
        <svg id="proposed" height="200" viewBox="0 0 100 100"></svg>
""")

HTML{String}("        <svg id=\"current\" height=\"200\" viewBox=\"0 0 100 100\"></svg>\n        <svg id=\"proposed\" height=\"200\" viewBox=\"0 0 100 100\"></svg>\n")

In [22]:
current_trace = Trace()
intervene!(current_trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(current_trace, "x$i", point.x)
    constrain!(current_trace, "y$i", point.y)
end
@generate(current_trace, agent_model())
current_score = score(current_trace)
for i=1:1000
    proposed_trace = deepcopy(current_trace)
    @generate(proposed_trace, agent_model(agent_model))
    proposed_score = score(proposed_trace)
    if log(rand()) < proposed_score - current_score
        current_trace = proposed_trace
        current_score = proposed_score
    end
    render(current_trace_renderer, current_trace)
    render(proposed_trace_renderer, proposed_trace)
    sleep(0.1)
end

LoadError: MethodError: no method matching agent_model(::Gen.Trace, ::#agent_model)[0m
Closest candidates are:
  agent_model(::Gen.AbstractTrace) at In[3]:4[0m

# 4. Improving the model

Our model above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that does not match our model's expectations. Let's see what happens when we try to do probabilistic inference in our model, given this data.

In [23]:
# show results (they should look bad)

This is an example of *model mis-specification*, which is when the model is not a good match for the distribution of data we are analyzing.

We can improve the model by adding the possibliltiy thathte agent takes a detour. Specifically, we add the possibility that the agent uses a waypoint and first walks from the starting locatoin to the waypoint and then from the waypoit to the final destination. Let's visualize the resulting samples:

In [24]:
# render_grid()...

Now, we can run inference as before:

In [25]:
# Do SIR with the new dataset and the improved model, and the same number of particles we used above.

In [26]:
# results for the same number of particles as above should look bad.

It looks like the results are not accurate. This is because in the new model, a random forward execution of the imporved model is a lot less likely to match the observations than a random forward exection of the original program. We can try to increase the number of samples to incrase the probability that we get one that matches the data. Note that this will take a few minutes t orun:

In [27]:
# Show results with a larger number of particles (should take < 1 min, should look okay)

This modification of the model mad einference a lot more computationally challenging, and our importance sampling algorihtm is not able to give us real-time inferneces. This motivates the need for a more sophisicated approach to probabilisitc inference.

# 5. Compiling inference with neural networks

There are a number of approaches for creating more efficient inference algorithms. We will focus on one approach, where we train a neural network to make informed guesses about he locatio nof the waypoint. First, let's understand in a bit more detail why the default inference algorihtm was slow

Suppose we knew the right waypoint:

In [28]:
trace = Trace()
constrain!(trace, "use-waypoint", true)
constrain!(trace, "waypoint", Point(0.5, 0.5))

Point(0.5,0.5)

Then the baseline importance sapmling algorithm gives reasonable inferences with fewer samples. We use this idea by training a neural network to make informed guesses about the waypoint, given the observed data as its input. We train the neural network on nany simulatoins of the program. Then, the resulting trained neural network can be used to make informed guesses about the waypooint given any observations.

In [29]:
# show the neural network, and show the training.

Let's visualize the guesses made by the neural network for a few  different datasets:

In [30]:
# show four renderings left to right of different datasets, with circles denoting the neural network's guess about the waypoint. 

Now, let's use this trained neural network to speed up inference:

In [31]:
# show the modififed SIR algorithm, using propose!

In [32]:
# show results for fewer particles, which should be noticeably faster than without the neural network.
# make an explicit comparison.